## AutoEncoder

In [6]:
# Interactive Plotting
%matplotlib notebook 

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import sys

# Import MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/")

# Training Constants
learning_rate = 0.005
batch_size = 128
train_steps = 2000

# Architecture
d = 28**2
h = 10

X = tf.placeholder("float", [None, d])
L = tf.placeholder("float", [None, h])

var_init = tf.contrib.layers.xavier_initializer()

w_enc = tf.Variable(var_init( [d, h] ))
b_enc = tf.Variable(var_init( [h] ))
w_dec = tf.Variable(var_init( [h, d] ))
b_dec = tf.Variable(var_init( [d] ))

enc_X = tf.nn.relu(tf.matmul(X, w_enc) + b_enc)
dec_X = tf.nn.relu(tf.matmul(enc_X, w_dec) + b_dec)

dec = tf.nn.relu(tf.matmul(L, w_dec) + b_dec) # for use to later decoding latent codes. 

loss = tf.reduce_mean(tf.pow( X - dec_X, 2))
adam = tf.train.AdamOptimizer(learning_rate)
optimizer = adam.minimize(loss)

comp_grad = adam.compute_gradients(loss, var_list=[w_enc, b_enc, w_dec, b_dec])

print(comp_grad)
placeholder_lst = []
print("")
for i in range(4):
    tupple = comp_grad[i]
    g = tupple[0]
    v = tupple[1]
    print(g.dtype, g.shape, v.dtype, v.shape)
    print("")
    
    placeholder_g = tf.placeholder(g.dtype, g.shape)
    placeholder_v = tf.placeholder(v.dtype, v.shape)
    new_tupple = (placeholder_g, placeholder_v)
    
    placeholder_lst.append( new_tupple )
    
print(placeholder_lst)

upda_grad = adam.apply_gradients(placeholder_lst)

init = tf.global_variables_initializer()


# initialize interactive plotting
fig, (ax_train, ax_score, ax_done) = plt.subplots(1, 3, figsize=(12,4)) 
fig.tight_layout()
ax_train.axis("off")
ax_done.axis("off")
ax_score.set_xlim(0, train_steps)
visualize, _ = mnist.test.next_batch(4) 
loss_vals = []

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(1, train_steps):
        
        batch, _ = mnist.train.next_batch(batch_size)
        #_, current_loss = sess.run([optimizer, loss], feed_dict={X: batch})
        current_loss = sess.run(loss, feed_dict={X: batch})
        
        
        S = sess.run(comp_grad, feed_dict={X: batch})
        
        
        sess.run(upda_grad, feed_dict={placeholder_lst: S})
        
        
        
        #sess.run(upda_grad, feed_dict={X: batch})
        #print(len(S))
        #print(S)
        #print(np.shape(S))
        
        #sess.run(upda_grad, ?? feed_dict.. S)
        
        loss_vals.append(current_loss)
        
        #grad_ = sess.run(grad, feed_dict={X: batch})
        #tf.gradients(dec_X, X, feed_dict={X: batch})
        
        
        # Visualize cool stuff
        if i % 100 == 0 or i == 1:  
            
            ax_score.plot(loss_vals, color='b')
            fig.canvas.draw()  
            
            pred, current_loss = sess.run([dec_X, loss], feed_dict={X: visualize})
            
            sys.stdout.write("\rStep: %i\tLoss: %f" % (i, current_loss))
            sys.stdout.flush() 
            
            images = np.empty((28 * 2, 28 * 2 * 2)) # add 2px space in between?
    
            for i in range(2):
                for j in range(2):
                    current_image = j + i*2

                    real_image = visualize[current_image].reshape([28, 28])
                    reconstructed_image = pred[current_image].reshape([28, 28])

                    # Add original image
                    images[i * 28: (i + 1) * 28, j * 28:(j + 1) * 28] = real_image

                    # Add reconstructed image
                    images[i * 28: (i + 1) * 28, j * 28+ 28*2:(j + 1) * 28+28*2] = reconstructed_image

            ax_train.imshow(images, origin="upper", cmap="gray", aspect='auto')
            fig.canvas.draw()      
        
    # print real images vs reconstructed images
    batch, _ = mnist.test.next_batch(100) # !! NOTICE WE TAKE TEST DIGITS TO ENSURE NOT OVERFIT!
    pred = sess.run(dec_X, feed_dict={X: batch})
    
    c = 6 
    images = np.empty((28 * c, 28 * c * 2)) # add 2px space in between?
    
    for i in range(c):
        for j in range(c):
            current_image = j + i*c
            
            real_image = batch[current_image].reshape([28, 28])
            reconstructed_image = pred[current_image].reshape([28, 28])
            
            # Add original image
            images[i * 28: (i + 1) * 28, j * 28:(j + 1) * 28] = real_image
                
            # Add reconstructed image
            images[i * 28: (i + 1) * 28, j * 28+ 28*c:(j + 1) * 28+28*c] = reconstructed_image

      
ax_done.imshow(images, origin="upper", cmap="gray", aspect='auto')
fig.canvas.show()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
[(<tf.Tensor 'gradients_11/MatMul_15_grad/tuple/control_dependency_1:0' shape=(784, 10) dtype=float32>, <tf.Variable 'Variable_20:0' shape=(784, 10) dtype=float32_ref>), (<tf.Tensor 'gradients_11/add_15_grad/tuple/control_dependency_1:0' shape=(10,) dtype=float32>, <tf.Variable 'Variable_21:0' shape=(10,) dtype=float32_ref>), (<tf.Tensor 'gradients_11/MatMul_16_grad/tuple/control_dependency_1:0' shape=(10, 784) dtype=float32>, <tf.Variable 'Variable_22:0' shape=(10, 784) dtype=float32_ref>), (<tf.Tensor 'gradients_11/add_16_grad/tuple/control_dependency_1:0' shape=(784,) dtype=float32>, <tf.Variable 'Variable_23:0' shape=(784,) dtype=float32_ref>)]

<dtype: 'float32'> (784, 10) <dtype: 'float32_ref'> (784, 10)

<dtype: 'float32'> (10,) <dtype: 'float32_ref'> (10,)

<dtype: 'float32'> (10, 784) <dtype: 'float32_ref'> (1

NotImplementedError: ('Trying to optimize unsupported type ', <tf.Tensor 'Placeholder_29:0' shape=(784, 10) dtype=float32>)